In [1]:
%matplotlib inline
import seaborn
import requests
import json

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#check for latest available date of data per series 
def latest_available(seriesid):
    api_json_latest = fr'https://api.bls.gov/publicAPI/v2/{seriesid}/latest=true'
    request = requests.post(api_json_latest)
    latest_json = json.loads(request.text)['Results']['series']['data']
    request.close()
    return latest_json

def survey_metadata(survey):
    #add verification that survey exists (use survey_list_json)
    api_json_survey = fr'https://api.bls.gov/publicAPI/v2/surveys/{survey}'
    request = requests.post(api_json_survey)
    survey_metadata_json = json.loads(request.text)['Results']['survey']
    request.close()
    return survey_metadata_json

#create search function for dictionary pulls
#create list of all available surveys
def surveys():
    api_survey_list = r'https://api.bls.gov/publicAPI/v2/surveys'
    survey_request = requests.post(api_survey_list)
    survey_list_json = json.loads(survey_request.text)['Results']['survey']
    survey_request.close()
    return survey_list_json

#create dictionary of industries
def industry_dict():
    industry_table = pd.read_table(r'https://download.bls.gov/pub/time.series/oe/oe.industry', index_col = 'sort_sequence')
    industry_codes_json = json.loads(industry_table.to_json(orient='table'))['data']
    del industry_table 
    return industry_codes_json

#create dictionary of occupations
def occupation_dict():
    occupation_table = pd.read_table(r'https://download.bls.gov/pub/time.series/oe/oe.occupation', index_col = 'sort_sequence')
    occupation_codes_json = json.loads(occupation_table.to_json(orient='table'))['data']
    del occupation_table
    return occupation_codes_json

#create dictionary of areas
def area_codes():
    area_table = pd.read_table(r'https://download.bls.gov/pub/time.series/oe/oe.area', index_col = 'sort_sequencye')
    area_codes_json = json.loads(area_table.to_json(orient='table'))['data']
    del area_table
    return area_codes_json
    
#pull dictionary codes per survey

series_dict = {'Seasonal Adjustment': {'Seasonally Adjusted':'S', 
                                       'Unadjusted':'U' },
              'Area Type':{'Metropolitan or nonmetropolitan area':'M', 
                           'National':'N',
                           'State':'S'},
               'Area':'',
               'Industry':'',
               'Occupation':'',
               'Datatype Code':{'Employment':'01',
                               'Employment percent relative standard error':'02',
                               'Hourly mean wage':'03',
                               'Annual mean wage':'04',
                               'Wage percent relative standard error':'05',
                               'Hourly 10th percentile wage':'06',
                               'Hourly 25th percentile wage':'07',
                               'Hourly median wage':'08',
                               'Hourly 75th percentile wage':'09',
                               'Hourly 90ths percentile wage':'10',
                               'Annual 10ths percentile wage':'11',
                               'Annual 25th percentile wage':'12',
                               'Annual median wage':'13',
                               'Annual 75th percentile wage':'14',
                               'Annual 90ths percentile wage':'15',
                               'Employment per 1,000 jobs':'16',
                               'Location Quotient':'17'
                               },
                }


In [76]:
def series_builder(prefix = 'OE',
                   seasonal_adjustment_code = 'U',
                   area_type_code = 'N',
                   area_code = '0000000',
                   industry_code = '000000',
                   occupation_code = '000000',
                   data_type_code = '01'):
    #add verficiation steps to ensure codes are within limits
    series = prefix + seasonal_adjustment_code + area_type_code + area_code + industry_code + occupation_code + data_type_code
    return series

def series_call(series, key, startyear = '', endyear = '', catalog = 'False', calculations = 'False', annualaverage = 'False', aspects = 'False'):
    api_json = r'https://api.bls.gov/publicAPI/v2/timeseries/data/'
    api_xlsx = r'https://api.bls.gov/publicAPI/v2/timeseries/data.xlsx'
    header = {'Content-type': 'application/json'}

    payload = json.dumps({'seriesid':series,
                         'startyear':startyear,
                         'endyear':endyear,
                         'catalog':catalog,
                         'calculations':calculations,
                         'annualaverage':annualaverage,
                         'aspects':aspects,
                         'registrationkey':key})
    request = requests.post(api_json, data = payload, headers = header)
    json_data = json.loads(request.text)['Results']['data']
    return json_data
    

In [18]:
#insert BLS API key here
key = ''